<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/17.%20Chapter17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 17: Representation Learning and Generative Learning using Autoencoder and GAN

Bab ini membahas dua arsitektur jaringan saraf yang kuat untuk pembelajaran tanpa pengawasan (*unsupervised learning*): **Autoencoders** dan **Generative Adversarial Networks (GANs)**. Keduanya mampu mempelajari representasi data yang padat dan efisien (*representation learning*) dan beberapa di antaranya dapat menghasilkan data baru yang realistis (*generative learning*).

* **Autoencoders**:
    * **Konsep Inti**: Sebuah ANN yang dilatih untuk menyalin inputnya ke outputnya. Ia terdiri dari dua bagian: **encoder** yang mengompresi input menjadi representasi laten (*latent representation* atau *coding*), dan **decoder** yang merekonstruksi input dari *coding* tersebut. Dengan memberikan batasan (misalnya, membuat lapisan *coding* lebih kecil dari input), autoencoder dipaksa untuk belajar fitur-fitur penting dari data.
    * **Jenis-jenis Autoencoder**:
        * **Stacked Autoencoder**: Autoencoder dengan beberapa *hidden layer*, biasanya simetris.
        * **Convolutional & Recurrent Autoencoder**: Menggunakan lapisan konvolusional atau rekuren, cocok untuk data gambar atau sekuensial.
        * **Denoising Autoencoder**: Dilatih untuk merekonstruksi input yang bersih dari input yang sudah diberi *noise*. Ini memaksa model untuk belajar fitur yang lebih kuat.
        * **Sparse Autoencoder**: Diberi penalti jika lapisan *coding*-nya terlalu aktif, mendorong setiap neuron untuk merepresentasikan fitur yang spesifik dan berguna.
        * **Variational Autoencoder (VAE)**: Autoencoder **probabilistik** dan **generatif**. Alih-alih memetakan input ke satu titik di ruang laten, encoder VAE memetakan input ke sebuah distribusi probabilitas (biasanya Gaussian). Ini memungkinkan kita untuk mengambil sampel dari ruang laten untuk menghasilkan data baru yang terlihat realistis.

* **Generative Adversarial Networks (GANs)**:
    * **Konsep Inti**: GAN terdiri dari dua jaringan saraf yang saling bersaing:
        1.  **Generator**: Mencoba membuat data palsu yang terlihat nyata (misalnya, gambar wajah palsu).
        2.  **Discriminator**: Mencoba membedakan antara data asli dan data palsu yang dibuat oleh generator.
    * **Adversarial Training**: Selama training, generator dan diskriminator bermain game *zero-sum*. Generator menjadi lebih baik dalam "menipu" diskriminator, sementara diskriminator menjadi lebih baik dalam "menangkap" pemalsuan. Kompetisi ini mendorong kedua jaringan untuk menjadi lebih baik.
    * **Tantangan Training GAN**: Training GAN terkenal sulit karena dinamikanya yang tidak stabil. Masalah umum termasuk:
        * **Mode Collapse**: Generator hanya menghasilkan variasi data yang sangat terbatas (misalnya, hanya satu jenis wajah).
        * **Konvergensi yang Tidak Stabil**: Proses training bisa tiba-tiba menyimpang.
    * **Arsitektur GAN Lanjutan**:
        * **Deep Convolutional GAN (DCGAN)**: Memberikan panduan arsitektur untuk membangun GAN yang stabil menggunakan lapisan konvolusional.
        * **Progressive GANs**: Melatih GAN dengan cara menghasilkan gambar beresolusi rendah terlebih dahulu, lalu secara bertahap meningkatkan resolusinya.
        * **StyleGANs**: Arsitektur canggih yang memisahkan "gaya" (*style*) dari konten gambar, memungkinkan kontrol yang lebih baik dan menghasilkan gambar yang sangat realistis.

### 1. Stacked Autoencoder dengan Keras
Contoh sederhana untuk mengompresi dan merekonstruksi gambar Fashion MNIST.

```python
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Memuat dan mempersiapkan data Fashion MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train = X_train_full.astype(np.float32) / 255
X_valid = X_train[:5000]
X_train = X_train[5000:]

# Membangun stacked autoencoder
stacked_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"), # Lapisan coding
])

stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])

stacked_ae.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=1.0))

# history = stacked_ae.fit(X_train, X_train, epochs=20,
#                          validation_data=(X_valid, X_valid))
print("Stacked autoencoder telah dibuat dan dikompilasi.")
```

### 2. Variational Autoencoder (VAE) untuk Menghasilkan Gambar
VAE adalah model generatif. Setelah dilatih, kita bisa menggunakannya untuk membuat gambar baru.

```python
# Lapisan kustom untuk sampling
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return tf.random.normal(tf.shape(log_var)) * tf.exp(log_var / 2) + mean

codings_size = 10

# Membangun encoder VAE
inputs = keras.layers.Input(shape=[28, 28])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(150, activation="selu")(z)
z = keras.layers.Dense(100, activation="selu")(z)
codings_mean = keras.layers.Dense(codings_size)(z)       # μ
codings_log_var = keras.layers.Dense(codings_size)(z)    # γ = log(σ^2)
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = keras.Model(
    inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

# Membangun decoder VAE
decoder_inputs = keras.layers.Input(shape=[codings_size])
x = keras.layers.Dense(100, activation="selu")(decoder_inputs)
x = keras.layers.Dense(150, activation="selu")(x)
x = keras.layers.Dense(28 * 28, activation="sigmoid")(x)
outputs = keras.layers.Reshape([28, 28])(x)
variational_decoder = keras.Model(inputs=[decoder_inputs], outputs=[outputs])

# Menggabungkan menjadi VAE
_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = keras.Model(inputs=[inputs], outputs=[reconstructions])

# Menambahkan latent loss
latent_loss = -0.5 * tf.reduce_sum(
    1 + codings_log_var - tf.exp(codings_log_var) - tf.square(codings_mean),
    axis=-1)
variational_ae.add_loss(tf.reduce_mean(latent_loss) / 784.0)

variational_ae.compile(loss="binary_crossentropy", optimizer="rmsprop")

# history = variational_ae.fit(X_train, X_train, epochs=25, batch_size=128,
#                                validation_data=(X_valid, X_valid))
print("\nVariational Autoencoder (VAE) telah dibuat.")
```
Setelah melatih VAE, Anda dapat memanggil `variational_decoder.predict()` dengan input acak dari distribusi normal untuk menghasilkan gambar baru.

### 3. Generative Adversarial Network (GAN)
Contoh GAN sederhana untuk Fashion MNIST. Training GAN memerlukan *custom training loop*.

```python
# Membangun generator dan diskriminator
codings_size = 30
generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")
])
gan = keras.models.Sequential([generator, discriminator])

# Mengompilasi model
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False # Penting: bekukan diskriminator saat melatih generator
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

# Custom Training Loop (kerangka)
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        # print(f"Epoch {epoch + 1}/{n_epochs}")
        for X_batch in dataset:
            # Fase 1: Melatih diskriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)

            # Fase 2: Melatih generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        # Tampilkan hasil atau simpan gambar yang dihasilkan di akhir setiap epoch

# batch_size = 32
# dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
# dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)
# train_gan(gan, dataset, batch_size, codings_size)
print("\nGenerative Adversarial Network (GAN) telah dibuat.")
```
Training GAN sangat sensitif dan mungkin memerlukan banyak *tuning*. Kode di atas menunjukkan logika dasar dari proses training-nya.
